In [300]:
import matplotlib.pyplot as plt  
# iscrtavanje slika i plotova unutar samog browsera
%matplotlib inline 

import matplotlib.pylab as pylab
# prikaz vecih slika 
pylab.rcParams['figure.figsize'] = 21,15

import numpy as np
import cv2 # OpenCV biblioteka

def show_in_window_and_below(img, below=True):
    if (below):
        plt.imshow(img, 'gray')
    cv2.imshow('image', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# current image extension
ext = '.JPG'

BLACK = 0 # black pixel's color
WHITE = 255 # white pixel's color

In [301]:
# runs calculations
def calculate_runs(img):
    runs = [[1] for x in xrange(img.shape[1])] # each column starts with 1 black pixel
    black_runs_flat, white_runs_flat = [], []
    for col in xrange(img.shape[1]): # iterate through all columns
        img[0,col] = 0 # PAINT THE FIRST PIXEL IN PREDEFINED COLOR, to make all columns start the same
        run_index = 0 # start the run
        for row in xrange(1, img.shape[0]): # for each pixel/row in current column
            if (img[row-1 ,col] != img[row, col]):  # if they are not the same,
                # memorize the old run in corresponding array
                if (run_index % 2 == 0): # black run
                    black_runs_flat.append(runs[col][run_index])
                else:
                    white_runs_flat.append(runs[col][run_index])
                # start a new run
                run_index += 1
                runs[col].append(0)
            runs[col][run_index] += 1     # add a pixel to the current run
        # the column ended ==> save the last run for the ended column
        if (run_index % 2 == 0): # black run
            black_runs_flat.append(runs[col][run_index])
        else:
            white_runs_flat.append(runs[col][run_index])
    return runs, black_runs_flat, white_runs_flat

### Line *thickness* and *spacings* (black and white runs) analysis

In [302]:
from collections import Counter

def calculate_line_thickness(black_flat):
    num_top = 4 # number of top/most common runs
    black_count = Counter(black_flat) # Counter({1: 3, 8: 1, 3: 1, ...})
    m_c_black = black_count.most_common(num_top)
    print 'Top', num_top, 'most_common_black_runs:', m_c_black
    
    m_c_black1 = m_c_black[0][0]
    if (len(m_c_black) >= 2):
        m_c_black2 = m_c_black[1][0]
        if (m_c_black1*3 < m_c_black2): # kind of a sanity check
            line_thickness = m_c_black1
        else:
            line_thickness = (m_c_black1 + m_c_black2) / 2.
    else:
        line_thickness = m_c_black1
    
    print '>>> Calculated line thickness:  ', line_thickness
    return line_thickness

def calculate_line_spacing(white_flat, image_height):
    num_top = 4 # number of top/most common runs
    white_count = Counter(white_flat) # print white_count.most_common(50)
    m_c_white = white_count.most_common(num_top)
    print 'Top', num_top, 'most common white runs', m_c_white
    
    m_c_white1 = m_c_white[0][0]
    if (len(m_c_white) >= 2):
        m_c_white2 = m_c_white[1][0]
        
        if (m_c_white1 > image_height*0.5): # sanity check
            line_spacing = m_c_white2
        else:
            if (m_c_white2 > image_height*0.5):
                line_spacing = m_c_white1
            else:
                line_spacing = (m_c_white1 + m_c_white2) / 2.
    else:
        line_spacing = m_c_white[0][0]
    print '>>> Calculated line spacing: ', line_spacing
    return line_spacing

In [303]:
def remove_staff_lines(img, runs, line_height, staff_thickness_multiplier):
    # copy the image.. python is pass-by-object-reference so it is necessary!
    p = img.copy() # pass-by-object-reference: https://stackoverflow.com/a/33066581/2101117
    # NOTE: copying is NOT NECESSARY if we won't use the passed `img` after this function returns
    #edit the image
    for c in xrange(len(runs)):        # for every column
        cumulative = 0 # initialize the number of passed pixels
        for r in xrange(len(runs[c])): # for every run
            run_length = runs[c][r]
            if (r % 2 == 0): # black runs # every black run longer than 2 * line_height is deleted/whitened
                if (run_length < line_height * staff_thickness_multiplier):
                    # ++ AKO JE SLJEDEĆI/PRETHODNI %% BIJELI %% RUN = VISINA PRAZNINE +-1
                    # ++ AKO JE SLJEDEĆI/PRETHODNI %%  CRNI  %% RUN = VISINA LINIJE +-1
                    p[cumulative:cumulative + run_length, c] = [255]*(run_length)
            #else: # white runs
            #    do something maybe ?
            cumulative += run_length
    return p

In [304]:
def remove_staff_lines_with_lines_only(img, runs, line_height, staff_thickness_multiplier, lines_only_img):
    # copy the image.. python is pass-by-object-reference so it is necessary!
    p = img.copy() # pass-by-object-reference: https://stackoverflow.com/a/33066581/2101117
    # NOTE: copying is NOT NECESSARY if we won't use the passed `img` after this function returns
    #edit the image
    for c in xrange(len(runs)):        # for every column
        cumulative = 0 # initialize the number of passed pixels
        for r in xrange(len(runs[c])): # for every run
            run_length = runs[c][r]
            if (r % 2 == 0): # black runs # every black run longer than 2 * line_height is erased
                if (run_length < line_height * staff_thickness_multiplier):
                    for compare_px in xrange(cumulative, cumulative + run_length):
                        if (lines_only_img[compare_px, c] == BLACK):
                            p[compare_px, c] = 255
                        # p[cumulative:cumulative + run_length, c] = [255]*(run_length)
            cumulative += run_length
    return p

< # # # #  # # # #  # # # #  # # # #  # # # #  # # # >
### (RLE) run-based approach to  LOCATING  STAVES
< # # # #  # # # #  # # # #  # # # #  # # # #  # # # >

In [1]:
# FOR ONE STAVE -- Y_location
def print_staff_locations_y(staff_locations):
    staves_keylist = staff_locations.keys()
    staves_keylist.sort()
    print '\nSORT %d staves and their lines by y-coordinates' % (len(staff_locations))
    # print staff_locations
    for staves_key in staves_keylist:
        print "  STAVE AT Y = %s" % (staves_key)
        staff = staff_locations[staves_key][1]
        staff_keylist = staff.keys()
        staff_keylist.sort()
        for staff_key in staff_keylist:
            print '\tstaff element %s: %s' % (staff_key, staff[staff_key])
        print ''

# FOR MULTIPLE STAVES --- X_locations
def print_staff_locations_x(staff_locations_x):
    print 'staff_locations', staff_locations_x
    if staff_locations_x == {}:
        print 'Staff locations are EMPTY! :O'
        return -1
    
    staff_locations_x_keys = staff_locations_x.keys()
    staff_locations_x_keys.sort()
    print '\n----\nSORT %d staff location dictionaries' % (len(staff_locations_x))
    for staff_loc_x_key in staff_locations_x_keys:
        print '----\nStaves for x =', staff_loc_x_key
        staff_locations = staff_locations_x[staff_loc_x_key]
        print_staff_locations_y(staff_locations)
    return 0

# Adds "help" lines and spaces (eng. "ledger lines/spaces") to `locaions` dictionary
def add_helper_spaces_and_lines(locations, line_thickness, line_spacing, cumulative):
    line_thickness = int(round(line_thickness)) # round(2.4) = 2 ||| round(2.6) = 3  :)
    line_spacing = int(round(line_spacing))
    
    help_space_3_up_y = cumulative - (2 * line_thickness) - (3 * line_spacing) 
    locations[help_space_3_up_y] = (0, line_spacing)
    help_line_2_up_y = cumulative - (2 * line_thickness) - (2 * line_spacing)
    locations[help_line_2_up_y] = (1, line_thickness)
    help_space_2_up_y = cumulative - (1 * line_thickness) - (2 * line_spacing)
    locations[help_space_2_up_y] = (2, line_spacing)
    help_line_1_up_y = cumulative - (1 * line_thickness) - (1 * line_spacing)
    locations[help_line_1_up_y] = (3, line_thickness)
    help_space_1_up_y = cumulative - (0 * line_thickness) - (1 * line_spacing)
    locations[help_space_1_up_y] = (4, line_spacing)
    
    lowest_line = max(locations.keys()) # MAX because the lower the LINE, the bigger the Y-coordinate
    lowest_line_end = lowest_line + locations[lowest_line][1]
    
    help_space_1_down_y = lowest_line_end + (0 * line_thickness) + (0 * line_spacing)
    locations[help_space_1_down_y] = (14, line_spacing)
    help_line_1_down_y = lowest_line_end + (0 * line_thickness) + (1 * line_spacing)
    locations[help_line_1_down_y] = (15, line_thickness)
    help_space_2_down_y = lowest_line_end + (1 * line_thickness) + (1 * line_spacing)
    locations[help_space_2_down_y] = (16, line_spacing)
    help_line_2_down_y = lowest_line_end + (1 * line_thickness) + (2 * line_spacing)
    locations[help_line_2_down_y] = (17, line_thickness)
    help_space_3_down_y = lowest_line_end + (2 * line_thickness) + (2 * line_spacing) 
    locations[help_space_3_down_y] = (18, line_spacing)
    return 0 # everything is OK

def run_is_spacing_candidate(run_length, line_spacing):
    return (line_spacing * 0.85 <= run_length <= line_spacing * 1.15)

def run_is_line_candidate(run_length, line_thickness):
#     lower_bound = line_thickness * 0.4 if line_thickness > 3.6 else 1 # Helps with thin average lines
    return (line_thickness * 0.4 <= run_length <= line_thickness * 1.6)
    
# Checks if there is a staff and returns a tuple (found, start_pixel, locations, end_pixel)
#   1.1) True,  or  False
#   1.2) The y-coordinate of the stave's START (start_pixel),  or  -1
#   1.3) Dictionary of pairs: (line_or_space_start_y, (line_space_CODE, line_thickness))
#   1.4) The y-coordinate of the stave's END (end_pixel),  or  -1
def get_staff_with_spaces(run_index, runs, image_column, cumulative, line_thickness, line_spacing):
    # initialize the return dictionary with a line number 1, which has the CODE=5. Why 5? See the next line
    # >> help_space3_up (CODE=0), help_line2_up, help_space2_up, help_line1_up, help_space1_up, FIRST_LINE
    locations = {cumulative: (5, runs[run_index])} # first line starts at the current index
    line_space_counter = 6 # first line is saved, now go for other SPACES AND LINES
    # cumulative -> how low (Y) are we in this image column - Make_first_row_black_in_runs_calc idea will be discarded!
    end_pixel = cumulative + runs[run_index] # determines the y-coordinate where this the last-found element end
    stave_found = True
    
    for i in range(1, 9):
        run_length = runs[run_index + i]
        if i%2 == 1: # i is odd for line_spacings, since for i=0 we get first LINE's index (run_index)
            if not run_is_spacing_candidate(run_length, line_spacing):
                stave_found = False
                break
        else: # i%2 == 0
            if not run_is_line_candidate(run_length, line_thickness):
                stave_found = False
                break
        # we came to execute this line, so this run (space/line) is ok
        locations[end_pixel] = (line_space_counter, run_length) # add the SPACE/LINE
        line_space_counter += 1 # look for the next SPACES AND LINES
        end_pixel += run_length # advance down the column
    
    if not stave_found:
        return False, -1, {}, -1 # not_found, no_start, no_locations, no_end
    
    # prepare the return values
    add_helper_spaces_and_lines(locations, line_thickness, line_spacing, cumulative)
    start_pixel = min(locations.keys())
    # get the last line's y_coordinate and add her height to it -- that's where this stave ends
    lowest_space = max(locations.keys()) # MAX because the lower the SPACE, the bigger the Y-coordinate
    end_pixel = lowest_space + locations[lowest_space][1] # lowest SPACE at `lowest_space`, height: locations[lowest_space][1]
    return True, start_pixel, locations, end_pixel

# IS beginning of the first stave close to the beginning of the second?
def staves_are_close(stave1, stave2, line_thickness):
    return abs(stave1[1][0] - stave2[1][0]) < line_thickness

def find_staves_in_runs(runs, image_column, line_thickness, line_spacing):
    run_index = 0 # current run index
    cumulative = 0 # how many pixels did we pass - for getting run's color# The paint_the_first_row_black idea will be discarded!!!
    
    staff_counter = 0
    staves = {}
    while (run_index < len(runs)):# WHILE, because we need more control over the index vlue
        run_length = runs[run_index]
        if (image_column[cumulative] == BLACK and run_is_line_candidate(run_length, line_thickness)):
            staff_is_found, start_pixel, staff, end_pixel = get_staff_with_spaces(run_index, runs, image_column, cumulative, line_thickness, line_spacing)
            if (staff_is_found): # YEA!
                staff_counter += 1
                staves[start_pixel] = (staff_counter, staff, end_pixel)
                # A staff was found, so we need to go down 8 runs (4*spacing, 4*line) until the last line of this staff
                for r_i in range(run_index, run_index+9): # OR 8??
                    cumulative += runs[r_i] # print 'adding', runs[r_i], 'to cumulative'
                run_index += 8
            else: # staff was NOT found
                cumulative += run_length
        else:
            cumulative += run_length
        run_index += 1
    
    return len(staves), staves # return the size and the staves dictionary

# check that NO staves from the `staves` dictionary are close to this `stave`
def no_staves_are_close(staves, stave, staff_start):
    close_range = (stave[2] - staff_start)/2
    starts_list = range(staff_start - close_range, staff_start + close_range + 1) # + 1 because range() doesn't include the last element
    starts_list.remove(staff_start)
    intersection = [start for start in starts_list if start in staves]
    return intersection == [] # Return value could be a list of matches... examples below
    # EX_1: return [-3, 1, 5] if there are staves beginning 3px ABOVE, 1px BELOW, or 5px BELOW this staff
    # EX_2: return a list of staves that are close: [staves[start] for start in starts_list if start in staves]

### Check that staffs for each key (X coordinate) are counted well -- in the following code cell
     sheet music
    '''''''''''''
    |   TITLE   | If each dash (-) is one X value, then the
    | --------- | first 4 Xes will have 2 staves, and the 
    | ----      | remaining 5 will have only one.
    |___________| NOTE: staff numbers in dictionary tuples START FROM 1

In [310]:
# from collections import Counter  #  Counter module might be used for validation!

# mora bar jedan diff da bude manji od 
def any_keys_close_to(possibly_missing_key, smaller_dict, threshold=40):
    for key in smaller_dict:
        diff = abs(key - possibly_missing_key)
        if (diff <= threshold): # there IS a key close to the possibly_missing_key
            return True
    # We came here, so no keys close to the possibly_missing_key!
    return False

def get_keys_not_in_second_dict(bigger_dict, smaller_dict):
    missing_keys = []
    for possibly_missing_key in bigger_dict:
        if not(any_keys_close_to(possibly_missing_key, smaller_dict)):
            missing_keys.append(possibly_missing_key)
            print 'FOUND MISSING KEY', possibly_missing_key
    return missing_keys

def copy_staff_from_a_valid_dictionary(valid_x_staves_loc, invalid_x_staves_loc, printing_on=False):
    if (printing_on):
        print '\t:::::;;; PRINTING VALID AND INVALID STAVES_LOC DICTIONARIES ;;:::::::'
        print '\n ===== VALID ===='
        print_staff_locations_y(valid_x_staves_loc)
        print '\n ===== INVALID ===='
        print_staff_locations_y(invalid_x_staves_loc)
    
    # get the keys and sort them
    sorted_VALID_dict_keys = valid_x_staves_loc.keys()
    sorted_VALID_dict_keys.sort()
    
    sorted_INVALID_dict_keys = invalid_x_staves_loc.keys()
    sorted_INVALID_dict_keys.sort()
    
    
    # memorize their lenghts
    valid_keys_length = len(sorted_VALID_dict_keys)
    invalid_keys_length = len(sorted_INVALID_dict_keys)
    
    difference_in_key_lenght = valid_keys_length - invalid_keys_length
    if (difference_in_key_lenght > 0): # Invalid dictionary is missing some staves - more common case
        keys_not_present_in_invalid_dict = get_keys_not_in_second_dict(sorted_VALID_dict_keys, sorted_INVALID_dict_keys)
        print 'keys that INVALID dict is missing:', keys_not_present_in_invalid_dict
        for valid_missing_keys in keys_not_present_in_invalid_dict:
            invalid_x_staves_loc[valid_missing_keys] = valid_x_staves_loc[valid_missing_keys]
    else: # invalid dict has more "staves" ---- just delete the invalid values :D
        invalid_staffs_keys = []
        # SLICE IT!!!
        keys_not_present_in_valid_dict = get_keys_not_in_second_dict(sorted_INVALID_dict_keys, sorted_VALID_dict_keys)
        print 'keys that VALID dict is missing:', keys_not_present_in_valid_dict
        for invalid_key in keys_not_present_in_valid_dict:
            print 'deleting key', loc_key, '.. use my_dict.pop(key, None) to get and delete it'
            del invalid_x_staves_loc[invalid_key]
    
    # we need to sort the keys again, since their number changed
    sorted_INVALID_dict_keys = invalid_x_staves_loc.keys()
    sorted_INVALID_dict_keys.sort()
    
    print 'adjusting fixed dictionary\'s staff numbers...'
    staff_counter = 1 # staff numbers start from ONE
    for staff_key in sorted_INVALID_dict_keys:
        print 'Staff at y = %d had the number %d' % (staff_key, invalid_x_staves_loc[staff_key][0])
        # staves[start_pixel] = (staff_counter, staff, end_pixel)
        invalid_x_staves_loc[staff_key] = (staff_counter, invalid_x_staves_loc[staff_key][1], invalid_x_staves_loc[staff_key][2])
        print 'Now, the number is %d' % (invalid_x_staves_loc[staff_key][0])
        staff_counter += 1
    print '\t <<<< FIXING DONE! >>> '
    if (printing_on):
        print '\t:::::;;; PRINTING VALID AND INVALID STAVES_LOC DICTIONARIES ;;:::::::'
        print '\n ===== VALID ===='
        print_staff_locations_y(valid_x_staves_loc)
        print '\n ===== INVALID - NOW ALSO VALID ===='
        print_staff_locations_y(invalid_x_staves_loc)
    return 0

def get_invalid_x_stave_locations(x_stave_locations):
    print '\n~ ~ ~ ~ ~ ~\n  validating x_stave_locations....'
    invalid_x_stave_locations = [] # not used...
    sorted_x_staves_keys = x_stave_locations.keys()
    sorted_x_staves_keys.sort()
    lengths = [len(x_stave_locations[x_staves_key]) for x_staves_key in sorted_x_staves_keys]
    
    # initialize the currently valid length to the maximum length
    valid_lenght = max(lengths)
    print 'lengths: ', lengths, '| max:', valid_lenght
    
    index_of_key = 0
    for x_staves_key in sorted_x_staves_keys:
        current_x_staves_loc = x_stave_locations[x_staves_key]
        current_length = len(current_x_staves_loc)
        print 'checking key:', x_staves_key, 'of size', current_length
        
        if (current_length <= valid_lenght): # the number of staves can decrease (or stay the same)
            print 'current lenght goes to:', current_length
            valid_lenght = current_length
        else: # but if it INCREASES, it's probably an ERROR!
            print ' >> WARNING << number of staves for X = %s is BIGGER than the last one' % x_staves_key
            print ' >> trying to fix it...'
            # determine where is the error
            # We will fix it by copying the missing staves from a neighbour x_stave_location dictionary
            if (index_of_key - 1 > 0):
                one_x_staves_loc_before = x_stave_locations[sorted_x_staves_keys[index_of_key - 1]]
                if (index_of_key + 1 < len(sorted_x_staves_keys)): # one after
                    one_after_or_two_x_staves_loc_before = x_stave_locations[sorted_x_staves_keys[index_of_key + 1]]
                else: # if it's the last dictionary, go two dicts behind
                    if (index_of_key - 2 >= 0):
                        one_after_or_two_x_staves_loc_before = x_stave_locations[sorted_x_staves_keys[index_of_key - 2]]
                    else:
                        print ' >>WARNING<< size is equal to (2?) --->', len(sorted_x_staves_keys)
                        one_after_or_two_x_staves_loc_before = one_x_staves_loc_before # just do something.. :D
                        return []
                
                # if the two nearest neighbours are of even sizes, this dict is invalid!
                if len(one_x_staves_loc_before) == len(one_after_or_two_x_staves_loc_before):
                    print 'error found in this dictionary'
                    valid_x_staves_loc = one_x_staves_loc_before
                    invalid_x_staves_loc = current_x_staves_loc
                else: # We ASSUME that the invalid dictionary was the one BEFORE the current one
                    print 'INVALID is the one before this, with key', sorted_x_staves_keys[index_of_key - 1]
                    valid_x_staves_loc = current_x_staves_loc # We ASSUME that current dict is valid
                    invalid_x_staves_loc = x_stave_locations[sorted_x_staves_keys[index_of_key - 1]]
                # try to fix the problem
                copy_staff_from_a_valid_dictionary(valid_x_staves_loc, invalid_x_staves_loc)
            else: # The first dictionary is invalid :O # WE ASSUME THAT THE SECOND DICT IS VALID !!!
                print ' >> WARNING << first x_stave_location is shorter than the max | ', # comma--print to same line
                print '(%d < %d)' % (len(first_x_stave_location), valid_lenght)
                print 'unahndled case - watch for exceptions and check results! Women and children first! :D'
                return [sorted_x_staves_keys[0]]
        index_of_key += 1 # !!!
    
    return [] # "Everything is OK."

### Main method for locating staves  -  calls find_staves_in_runs (two cells above)

In [311]:
def locate_lines_with_runs(lines_only_img):
    print 'lines_only_img.shape', lines_only_img.shape
    img_height, img_width = lines_only_img.shape # unpack values
    
    # runs calculation, for thickness and spacing
    runs, black_runs_flat, white_runs_flat = calculate_runs(lines_only_img)
    line_thickness = calculate_line_thickness(black_runs_flat)
    line_spacing = calculate_line_spacing(white_runs_flat, img_height)
    
    # get the columns that we will check for lines  #  IDEA:  Parametrize this! -- described in  IDEJE.txt
    # columns = [img_width/5, img_width/5+1, img_width/5+2, img_width/3, img_width/3+1, img_width/3+2]
    columns_to_check = [img_width/5, img_width/4, img_width/3, img_width/2, img_width/3*2, img_width/4*3]
    # for a 2400px wide image, values are: 480 600 800 1200 1600 1800
    # ANOTHER APPROACH: [img_width/6 * i for i in range(1,6)] # 6 could be 7 or 8, for  MORE PRECISION !!
    print 'columns_to_check', columns_to_check
    
    search_width = 3 # how wide will we look arround each pixel - to BOTH sides!
    resulting_staves = {} # DICTIONARY!
    
    for col_to_check in columns_to_check:
        print '\n -- -- -- ~~ -- -- ~~ -- ~~ -- ~~\nchecking around column', col_to_check
        column_staves = resulting_staves[col_to_check] = {} # shorter name === column_staves
        
        start_column, end_column = col_to_check-search_width, col_to_check+search_width
        column_range = range(start_column, end_column + 1) # + 1 because range() doesn't include the last element
        print 'range(start_column, end_column)', column_range
        for col in column_range:
            print '\n ~~~~~~~~\n  checking column >>>', col, '<<<'
            image_column = lines_only_img[0:img_height, col].tolist() # convert numpy array to python list
            num_of_staves, staves = find_staves_in_runs(runs[col], image_column, line_thickness, line_spacing)
            if (num_of_staves > 0):
                for staff_start in staves: # iterate through keys # or something like that
                    # IF there is NO staffs that are close to this one (on y-axis)
                    if (no_staves_are_close(column_staves, staves[staff_start], staff_start)):
                        column_staves[staff_start] = staves[staff_start]
                # break # !!! no need to check arround the `col_to_check` any more - go to next part of the image
    
    invalid_x_stave_locations = get_invalid_x_stave_locations(resulting_staves)
    if (invalid_x_stave_locations == []):
        return resulting_staves
    return {}

In [312]:
def dilate_and_save(img, kernel_w, kernel_h):
    '''Dilates a binary image with kernel of specified dimensions.
    saves the image to hard drive and returns the saved image'''
    kernel = np.ones((kernel_w, kernel_h), dtype=np.int)
    lines_only_img = cv2.dilate(img, kernel, iterations=1)
    lines_only_img_path = './images/locate_lines/dil_{}_{}.jpg'.format(kernel_w, kernel_h)
    cv2.imwrite(lines_only_img_path, lines_only_img)
    return lines_only_img

def erode_and_save(img, kernel_w_e, kernel_h_e, kernel_w_d=0, kernel_h_d=0):
    '''Erodes a binary image with kernel of specified dimensions.
    saves the image to hard drive and returns the saved image'''
    if (kernel_w_d == 0):
        dil_str = ''
    else:
        dil_str = '_dil_{}_{}'.format(kernel_w_d, kernel_h_d)
    kernel = np.ones((kernel_w_e, kernel_h_e), dtype=np.int)
    lines_only_img = cv2.erode(img, kernel, iterations=1)
    lines_only_img_path = './images/locate_lines/er_{}_{}{}.jpg'.format(kernel_w_e, kernel_h_e, dil_str)
    cv2.imwrite(lines_only_img_path, lines_only_img)
    return lines_only_img

### Two approaches on getting the lines-only image:
####  |__ 1: dilate and then erode with a very wide kernel (written right below this heading)
####  |__ 2: use the previous approach's image + remove all non-staff-line symbols (implementation started in 'testing stuff' notebook)

In [313]:
def get_lines_only_img(img):
    threshold = cv2.ADAPTIVE_THRESH_MEAN_C
    method, block, c = cv2.THRESH_BINARY, 55, 9 # 35, 13  || 45, 11
    img_ada = cv2.adaptiveThreshold(img, 255, threshold, method, block, c)

    # dilate with 1x50 --> erode with 2x50  | this has the best results
    kernel_w, kernel_h = 1, 50
    dilated = dilate_and_save(img_ada, kernel_w, kernel_h)
    kernel_w_e, kernel_h_e = 2, 50
    return erode_and_save(dilated, kernel_w_e, kernel_h_e, kernel_w, kernel_h)

def get_line_locations(img, print_result=True):
    '''Locate lines on the given image and print results (if print_results is `True`)
    Returns: staff lines LOCATIONS and the lines-only image'''
    lines_only_img = get_lines_only_img(img) # get_lines_only_img_CLEAR # approach 2 - maybe in the future
    
    # use the lines-only image to locate the staves
    staff_locations = locate_lines_with_runs(lines_only_img)
    
    if (print_result):
        print_staff_locations_x(staff_locations)
    return staff_locations
    
img = cv2.imread('./images/dataset/newest/bolujem.jpg', 0) # 0 --> read as grayscale
staf_locs = get_line_locations(img, False) # False ---> DON'T PRINT!

lines_only_img.shape (3264L, 2448L)
Top 4 most_common_black_runs: [(5, 25554), (6, 5450), (4, 3868), (1, 2448)]
>>> Calculated line thickness:   5.5
Top 4 most common white runs [(16, 19971), (17, 4490), (15, 3210), (313, 561)]
>>> Calculated line spacing:  16.5
columns_to_check [489L, 612L, 816L, 1224L, 1632L, 1836L]

 -- -- -- ~~ -- -- ~~ -- ~~ -- ~~
checking around column 489
range(start_column, end_column) [486, 487, 488, 489, 490, 491, 492]

 ~~~~~~~~
  checking column >>> 486 <<<
 ~~ get_staff_with_spaces ~~
  checking:
	 run_index, 2 / 32 	 runs: [1, 503, 6, 15, 5, 16, 5, 16, 5, 16, 6, 309, 5, 16, 5, 16, 5, 16, 5, 16, 5, 306, 5, 16, 4, 17, 4, 16, 5, 15, 6, 1878]

<<<< we got them linez! >>>
 ~~ get_staff_with_spaces ~~
  checking:
	 run_index, 12 / 32 	 runs: [1, 503, 6, 15, 5, 16, 5, 16, 5, 16, 6, 309, 5, 16, 5, 16, 5, 16, 5, 16, 5, 306, 5, 16, 4, 17, 4, 16, 5, 15, 6, 1878]

<<<< we got them linez! >>>
 ~~ get_staff_with_spaces ~~
  checking:
	 run_index, 22 / 32 	 runs: [1, 50

<<<< we got them linez! >>>
 ~~ get_staff_with_spaces ~~
  checking:
	 run_index, 22 / 32 	 runs: [1, 502, 5, 16, 4, 17, 4, 17, 5, 16, 5, 312, 6, 16, 5, 16, 5, 16, 5, 16, 5, 308, 5, 16, 5, 16, 5, 16, 5, 15, 6, 1873]

<<<< we got them linez! >>>
close_range = ( 655 - 440 ) / 2
close_range = ( 1057 - 841 ) / 2
close_range = ( 1454 - 1239 ) / 2

 ~~~~~~~~
  checking column >>> 1224 <<<
 ~~ get_staff_with_spaces ~~
  checking:
	 run_index, 2 / 32 	 runs: [1, 502, 5, 16, 4, 17, 4, 17, 5, 16, 5, 312, 6, 16, 5, 16, 5, 16, 5, 16, 5, 308, 5, 16, 5, 16, 5, 16, 5, 15, 6, 1873]

<<<< we got them linez! >>>
 ~~ get_staff_with_spaces ~~
  checking:
	 run_index, 12 / 32 	 runs: [1, 502, 5, 16, 4, 17, 4, 17, 5, 16, 5, 312, 6, 16, 5, 16, 5, 16, 5, 16, 5, 308, 5, 16, 5, 16, 5, 16, 5, 15, 6, 1873]

<<<< we got them linez! >>>
 ~~ get_staff_with_spaces ~~
  checking:
	 run_index, 22 / 32 	 runs: [1, 502, 5, 16, 4, 17, 4, 17, 5, 16, 5, 312, 6, 16, 5, 16, 5, 16, 5, 16, 5, 308, 5, 16, 5, 16, 5, 16, 5, 15, 6,


<<<<  Pitch recognition  >>>>
---

### In MusicXML, scientific pitch notation is used  (see <a href="https://en.wikipedia.org/wiki/Scientific_pitch_notation"> wikipedia page about Scientific pitch notation</a>)
Therefore, we used it too.

In [314]:
# We currently support notes
# from: above 2nd UPPER ledger/helper line
# to:   below 2nd LOWER ledger/helper line
clefs = {
  'treble': ['D6','C6',  'B5','A5','G5','F5','E5','D5','C5',  'B4','A4','G4','F4','E4','D4','C4',  'B3','A3','G3'],
  'bass': ['F4','E4','D4','C4',  'B3','A3','G3','F3','E3','D3','C3',  'B2','A2','G2','F2','E2','D2','C2',  'B1']
}

# some of the "important" element codes
HIGHEST_SPACING = 0 #   highest ledger/helper spacing
FIRST_LINE = 5      # first full-width staff-line
FIFTH_LINE = 13     # last full-width staff-line
LOWEST_SPACING = 18 #   lowest ledger/helper spacing

### Obtain the closest / corresponding staff

In [315]:
def get_closest_staff_locations_to_x(staff_locations_by_x, symbol_X_coordinate):
    sorted_loc_by_x_keys = staff_locations_by_x.keys()
    sorted_loc_by_x_keys.sort() # keys are now sorted # BUT NO NEED FOR SORTING IN THIS ALGORITHM !
    
    if (symbol_X_coordinate in sorted_loc_by_x_keys): # in case there is the same key as symbol_X_coordinate
        return symbol_X_coordinate
    
    chosen_key, min_diff = -1, 2000 # get the chosen_key  with  minimal difference from symbol_X_coordinate
    for x_key in sorted_loc_by_x_keys:
        diff = abs(x_key - symbol_X_coordinate)
        if (diff < min_diff):
            min_diff = diff
            chosen_key = x_key
    # print '\nclosest x_key for %d is %d' % (symbol_X_coordinate, chosen_key)
    return staff_locations_by_x[chosen_key]

# get the staff that contains the sent symbol (his y coordinate, actually)
# returns the matched staff and it's key (if return_staff_key is `True`) from staff_locations dictionary
def get_staff(symbol_y_coordinate, staff_locations, return_staff_key=False):
    # get staff locations keys (y-coordinates of staves beginnings)
    sorted_staff_locations_keys = staff_locations.keys()
    sorted_staff_locations_keys.sort() # make them sorted # NOTE :::: sorted_keys.reverse()  REVERSES THE SORT! (returns None!)
    
    # go through sorted keys in dictionary of staves (they are y-coordinates of each staff's beginning)
    # and choose only ones that are above the symbol's y-coordinate (i.e. their y-coordinate is a smaller number)
    candidates = [staff_key for staff_key in sorted_staff_locations_keys if (staff_key <= symbol_y_coordinate)]
    
#     print candidates, '| size:', len(candidates)
#     print '  smallest difference is between the lastly added candidate'
    chosen_staff_key = candidates[len(candidates) - 1]
#     print '  best match is:', chosen_staff_key
    
    # check that the symbol is valid, i.e. not lower than the chones staff's height
    # (exaple: if staff starts at 500px and ends at 700px and the symbol_y = 800px)
    if (staff_locations[chosen_staff_key][2] >= symbol_y_coordinate):
        if (return_staff_key):
            return staff_locations[chosen_staff_key], chosen_staff_key
        else:
            return staff_locations[chosen_staff_key]
    print 'symbol location is not valid!'
    return None

### Obtain the closest / corresponding staff-element's code

In [316]:
# returns the staff element's CODE: 0 for the 3rd upper spacing above the staff, ..., 5 for the 1st staff line
# diff ---> find the smallest difference, that is, find the closest y to the symbol_y_coordinate
def get_staff_element_code_diff(symbol_y_coordinate, staff):
#     print 'symbol_y:', symbol_y_coordinate, 'staff:', staff
    # get staff-line locations keys (y-coordinates of staff-lines beginnings)
    staff_elements = staff[1] # [1] ==> the 0th tuple element is staff's ordinal number
    staff_keys = staff_elements.keys()
    
    if (symbol_y_coordinate in staff_keys): # in case there is the same key as symbol_y_coordinate
        return staff_elements[symbol_y_coordinate][0] # just return the symbol for this key 
    
    chosen_key, min_diff = -1, 2000 # get the chosen_key  with  minimal difference from symbol_y_coordinate
    for y_key in staff_keys:
        diff = abs(y_key - symbol_y_coordinate)
        if (diff < min_diff):
            min_diff = diff
            chosen_key = y_key
    return staff_elements[chosen_key][0] # 0th tuple element is the CODE, 1st is element's height

# OLD
# returns the staff element's CODE: 0 for the 3rd upper spacing above the staff, ..., 5 for the 1st staff line
# def get_staff_element_code(symbol_y_coordinate, staff):
#     staff_elements = staff[1] # [1] ==> the 0th tuple element is staff's ordinal number
#     # get staff-line locations keys (y-coordinates of staff-lines beginnings)
#     sorted_staff_keys = staff_elements.keys()
#     sorted_staff_keys.sort() # make them sorted # NOTE :::: sorted_keys.reverse()  REVERSES THE SORT! (returns None!)
    
#     candidates = [staff_elem_key for staff_elem_key in sorted_staff_keys if (staff_elem_key <= symbol_y_coordinate)]
#     candidates_length = len(candidates)
#     chosen_line_key = candidates[candidates_length - 1]
    
#     element_code = staff_elements[chosen_line_key][0] # 0th tuple element is the CODE, 1st is element's height
    
#     # spacings are greedy, so we need to check if 
#     if (element_code % 2 == 0): # spacings have even codes
#         if (element_code != LOWEST_SPACING): # we are not at last spacing
#             next_element_key = sorted_staff_keys[candidates_length]
#             # check if next_element would be closer than the Chosen one (ha!)
#             if (abs(next_element_key - symbol_y_coordinate) < abs(chosen_line_key - symbol_y_coordinate)):
#                 print '  ! Chosen One was fake :O'
#                 print '  ! %d is closer to %d than %d ' % (next_element_key, symbol_y_coordinate, chosen_line_key)
#                 element_code = staff_elements[next_element_key][0]
#     return element_code

In [317]:
def get_pitch(staff_element_code, clef):
    return clefs[clef][staff_element_code]

def get_staff_and_pitch(symbol_x_y, staff_locations_by_x, clef='treble'):
    symbol_X, symbol_Y = symbol_x_y # unpack values for clairty and readability
    
    staff_locations = get_closest_staff_locations_to_x(staff_locations_by_x, symbol_X)
    
    staff = get_staff(symbol_Y, staff_locations)
    staff_number = staff[0] # first tuple element is staff's number (1,2,3...)
    
    staff_element_code_diff = get_staff_element_code_diff(symbol_Y, staff)
    pitch = get_pitch(staff_element_code_diff, clef) # pitch_diff
    print '   pitch for (x,y) = %s  ==> %s' % (symbol_x_y, pitch) # pitch_DIFF
    print '   staff number:', staff_number, '\n'
    return  staff_number, pitch
# old method for obtaining the staff_element_code
#     staff_element_code = get_staff_element_code(symbol_Y, staff)
#     pitch = get_pitch(staff_element_code, clef)
#     print '   pitch      for (x,y) = %s  ==> %s' % (symbol_x_y, pitch)

### test pitch-related functions

In [318]:
symbols_coordinates = [
    (1718, 1345), (271, 603), (2118, 527), (631, 1340), (2179, 1369), (1587, 1403), (1587, 1402), (1587, 1401)
] # ===> B4, D4, C5, B4, G4
# symbols_coordinates = [(231, 945), (231, 951), (231, 955), (231, 961), (231, 966), (231, 971)]
for symbol_x_y in symbols_coordinates:
    get_staff_and_pitch(symbol_x_y, staf_locs, clef='treble') # `clef` param for clarity :)

   pitch for (x,y) = (1718, 1345)  ==> B4
   staff number: 3 

   pitch for (x,y) = (271, 603)  ==> C4
   staff number: 1 

   pitch for (x,y) = (2118, 527)  ==> C5
   staff number: 1 

   pitch for (x,y) = (631, 1340)  ==> B4
   staff number: 3 

   pitch for (x,y) = (2179, 1369)  ==> G4
   staff number: 3 

   pitch for (x,y) = (1587, 1403)  ==> C4
   staff number: 3 

   pitch for (x,y) = (1587, 1402)  ==> C4
   staff number: 3 

   pitch for (x,y) = (1587, 1401)  ==> D4
   staff number: 3 



In [297]:
def last_test():
#     img = cv2.imread('./images/dataset/sviraj_up.jpg', 0) # read the image as GRAYcscale
    img_name = 'composicion'
    root_path = './images/dataset/' + img_name + '/'
    img = cv2.imread(root_path + img_name + '_up.jpg', 0) # read the image as GRAYcscale
    
    adaptiveMethod, thresholdType, blockSize, C = cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 55, 35
    img_ada = cv2.adaptiveThreshold(img, 255, adaptiveMethod, thresholdType, blockSize, C)
#     img_ada = cv2.erode(img_ada, np.ones((1,7), dtype=np.int)) # thickens the lines
    
    runs, black_runs_flat, white_runs_flat = calculate_runs(img_ada) # runs calculation, for thickness and spacing
    line_thickness = calculate_line_thickness(black_runs_flat)
    line_spacing = calculate_line_spacing(white_runs_flat, img_ada.shape[0]) # calculation needs image height
    
    lines_only_img = get_lines_only_img(img) # ...
    # line_thickness = 3 # CHEAT LINE
    erode_width = int(line_thickness+1) # set to be 4 allways?
    lines_only_img = cv2.erode(lines_only_img, np.ones((erode_width, 1), dtype=np.int)) # thickens the lines
    
    thickness_mul = 1.5 # staff_thickness_multiplier
#     thickness_mul = 2 # staff_thickness_multiplier
    
#     we can send the grayscale image, also :D  Just send `img` instead of `img_ada`
    result = remove_staff_lines_with_lines_only(img_ada, runs, line_thickness, thickness_mul, lines_only_img)
    result_path = root_path + 'RM_S_L_LINES_ONLY_b{}_c{}_ada{}_er_kw{}.jpg'.format(blockSize, C, adaptiveMethod, erode_width)
    cv2.imwrite(result_path, result)
    
#     runs, black_runs_flat, white_runs_flat = calculate_runs(img_ada) # runs calculation, for thickness and spacing
#     line_thickness = calculate_line_thickness(black_runs_flat)
#     line_spacing = calculate_line_spacing(white_runs_flat, img_ada.shape[0]) # needs image height
    
    thickness_mul = 1.5 # staff_thickness_multiplier
    
#     # Remove staff lines - RUNS algorithm ~~Simplest~~ (Aleksandar's)
#     rm_s_l = remove_staff_lines(img_ada, runs, line_thickness, thickness_mul)
#     path_regular = root_path + 'b_{}_c_{}_RM_S_L.jpg'
#     cv2.imwrite(path_regular.format(blockSize, C), rm_s_l)
    
#     # Remove staff lines - UP-DOWN algorithm (Aleksandar's)
#     rm_s_l_up_down = rm_staff_lines_up_down_neighbours(img_ada, runs, line_thickness, line_spacing, thickness_mul)
#     path_up_down = root_path + 'b_{}_c_{}_RM_S_L_UP_DOWN.jpg'
#     cv2.imwrite(path_up_down.format(blockSize, C), rm_s_l_up_down)
    
#     # Remove staff lines - SIDE algorithm (Filip's) # -----%%%%%%====%%%%  ADDED  LINES-ONLY-IMAGE %%%%%%%%==%%%==%%-
#     distance = int(line_spacing * 2.5)+1 ### check for: * 1, * 1.3, * 1.5
#     rm_s_l_side = rm_staff_lines_side_neighbours(img_ada, runs, line_thickness, line_spacing, thickness_mul, distance, lines_only_img)
#     path_side = root_path + 'ADA_b_{}_c_{}_RM_S_L_SIDE_dist_{}.jpg'
#     cv2.imwrite(path_side.format(blockSize, C, distance), rm_s_l_side)
    
#     distance = int(line_spacing * 1.7)+1 ### check for: * 1, * 1.3, * 1.5
#     rm_s_l_side = rm_staff_lines_side_neighbours(img_ada, runs, line_thickness, line_spacing, thickness_mul, distance, lines_only_img)
#     path_side = root_path + 'ADA_b_{}_c_{}_RM_S_L_SIDE_dist_{}.jpg'
#     cv2.imwrite(path_side.format(blockSize, C, distance), rm_s_l_side)

# last_test() # UNCOMMENT to start

In [85]:
qqq = {5:3} # 0 1 1 2 3
test_list = [6,7,8,9]
intersection = [i for i in test_list if i in qqq]
print(intersection)

[]
True
1.4 5.6


In [118]:
import os

def read_images(folder_path):
    if not (folder_path.endswith('/')):
        folder_path += '/'
    print os.listdir(folder_path)
    images = []
    image_names = os.listdir(folder_path)
    for image_name in image_names:
        images.append(cv2.imread(folder_path + image_name, 0)) # `0` --> read images as grayscale
    return image_names, images

# print len(read_images('./images/dataset/newest'))

In [ ]:
def process_images_in_folder(folder_path):
    image_names, images = read_images(folder_path)
    for i in range(len(images)):
        img = images[i]
        img_name = image_names[i]
        
        adaptiveMethod, thresholdType, blockSize, C = cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 55, 35
        
        img_ada = cv2.adaptiveThreshold(img, 255, adaptiveMethod, thresholdType, blockSize, C)
    #     img_ada = cv2.erode(img_ada, np.ones((1,7), dtype=np.int)) # thickens the lines

        runs, black_runs_flat, white_runs_flat = calculate_runs(img_ada) # runs calculation, for thickness and spacing
        line_thickness = calculate_line_thickness(black_runs_flat)
        line_spacing = calculate_line_spacing(white_runs_flat, img_ada.shape[0]) # calculation needs image height

        lines_only_img = get_lines_only_img(img) # ...
        # line_thickness = 3 # CHEAT LINE
        erode_width = int(line_thickness+1) # set to be 4 allways?
        lines_only_img = cv2.erode(lines_only_img, np.ones((erode_width, 1), dtype=np.int)) # thickens the lines

        thickness_mul = 1.5 # staff_thickness_multiplier
    #     thickness_mul = 2 # staff_thickness_multiplier

    #     we can send the grayscale image, also :D  Just send `img` instead of `img_ada`
        result = remove_staff_lines_with_lines_only(img_ada, runs, line_thickness, thickness_mul, lines_only_img)
        result_path = folder_path + img_name + 'RM_S_L_LINES_ONLY_b{}_c{}_ada{}_er_kw{}.jpg'
        cv2.imwrite(result_path.format(blockSize, C, adaptiveMethod, erode_width), result)
    return 0

process_images_in_folder('./images/dataset/newest/')

In [12]:
a = {1:2, -5: 3, 6:2}
sk = a.keys()
sk.sort() # returns None and `sk` becomes [-5, 1, 6]
sk.reverse() # returns None and `sk` becomes [6, 1, -5]

[6, 1, -5]


In [32]:
for ttt in range(0):
    print ttt

In [281]:
notes = ['D5', 'E3', 'C2', 'F1']
compare_to = 'B4'
for note in notes:
    print note[0] >= compare_to[0] and note[1] >= compare_to[1]

True
False
False
False
